In [ ]:
#importamos librerias
import pandas as pd

In [ ]:
#leemos el archivo para las consultas, en este caso usamos el que tiene el promedio de los scores
df=pd.read_parquet('/content/drive/MyDrive/plataformasConScore.parquet')

In [ ]:
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,duration_int,duration_type,score,platform
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,113.0,min,3.47,amazon prime
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2,110.0,min,3.55,amazon prime


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   show_id        22998 non-null  object        
 1   type           22998 non-null  object        
 2   title          22998 non-null  object        
 3   director       14739 non-null  object        
 4   cast           17677 non-null  object        
 5   country        11499 non-null  object        
 6   date_added     13444 non-null  datetime64[ns]
 7   release_year   22998 non-null  int64         
 8   rating         22998 non-null  object        
 9   duration       22811 non-null  object        
 10  listed_in      22998 non-null  object        
 11  description    22994 non-null  object        
 12  id             22998 non-null  object        
 13  duration_int   22811 non-null  float64       
 14  duration_type  22811 non-null  object        
 15  score          2299

In [ ]:
# El las columnas de duration_type hay nan y los llenamos con ceros,  y en el caso de duration_int ademas
# de llenar con ceros cambiamos el tipo de variable de float64 a int
df.duration_type = df.duration_type.fillna(0)
df.duration_int = df.duration_int.fillna(0).astype(int)

In [ ]:
#Con estos pasos volvemos a guardar por si se necesitan en los proximos pasos
df.to_csv('/content/drive/MyDrive/dfModificandoDuracion.csv', index=False) 

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/dfModificandoDuracion.csv')

In [ ]:
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,duration_int,duration_type,score,platform
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,113,min,3.47,amazon prime
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2,110,min,3.55,amazon prime


Revisamos algunas columnas para hacer algunas consideraciones en las consultas

In [ ]:
df.type.unique()

array(['movie', 'tv show'], dtype=object)

In [ ]:
df.release_year.min()

1920

In [ ]:
df.release_year.max()

2021

In [ ]:
df.score.min()

3.34

In [ ]:
df.score.max()

3.72

# Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. (la función debe llamarse get_max_duration(year, platform, duration_type))

In [ ]:
def get_max_duration(release_year=None, platform=None, duration_type=None):
    
    global df    
    
    # Verificar que el año esté dentro del rango válido
    if release_year is not None and (release_year <= 1920 or release_year >= 2021):
        raise ValueError("El año debe estar dentro del rango de 1920 a 2021.")

    # Verificar que la plataforma sea una de las opciones válidas
    if platform is not None and platform.lower() not in ['disney', 'amazon', 'hulu', 'netflix']:
        raise ValueError("La plataforma debe ser una de las opciones válidas: Disney, Amazon, Hulu o Netflix.")
   
    #Reemplazando porque en el data coloque los nombres completos de las plataformas
    if platform=='disney':
        platform='disney plus'
  
    if platform=='amazon':
        platform='amazon prime'

    # Verificar que el duracion_type esté dentro del rango válido
    if duration_type is not None and duration_type.lower() not in ['min', 'season', 'seasons']:
        raise ValueError("La duración debe ser una de las opciones válidas: min, season, seasons.")

    # Filtrar por películas únicamente
    df_soloPeliculas = df[df.type == 'movie']
    
    # Aplicar filtros opcionales si se proporcionan
    if release_year:
         df_soloPeliculas = df_soloPeliculas[df_soloPeliculas.release_year == release_year]
    if platform:
        df_soloPeliculas = df_soloPeliculas[df_soloPeliculas.platform == platform]
    if duration_type:
        df_soloPeliculas = df_soloPeliculas[df_soloPeliculas.duration_type == duration_type]

    # Verificar que hay al menos una película que cumpla con los filtros
    if df_soloPeliculas.empty:
        raise ValueError("No existe una película que cumpla con los filtros especificados.")

    # Ordenar por duración descendente y tomar la primera fila
    peliculaConMayorDuracion = df_soloPeliculas.sort_values('duration_type', ascending=False).iloc[0]

      

    detalles_peliculaConMayorDuracion = {
        'titulo': peliculaConMayorDuracion.title,
        'year': peliculaConMayorDuracion.release_year,
        'platform': peliculaConMayorDuracion.platform,
        'type': peliculaConMayorDuracion.type,
        'duracion': peliculaConMayorDuracion.duration_int
    }    
    return detalles_peliculaConMayorDuracion


In [ ]:
count_netflix = get_max_duration(2020, 'netflix', 'min')
print(count_netflix)

{'titulo': 'dick johnson is dead', 'year': 2020, 'platform': 'netflix', 'type': 'movie', 'duracion': 90}


# Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año (la función debe llamarse get_score_count(platform, scored, year))

In [ ]:
def get_score_count(platform=None, scored=None, year=None):
    
    global df    

    # Verificar que el año esté dentro del rango válido
    if year is not None and (year <= 1920 or year >= 2021):
        raise ValueError("El año debe estar dentro del rango de 1920 a 2021.")

    # Verificar que la plataforma sea una de las opciones válidas
    if platform is not None and platform.lower() not in ['disney', 'amazon', 'hulu', 'netflix']:
        raise ValueError("La plataforma debe ser una de las opciones válidas: Disney, Amazon, Hulu o Netflix.")
   
    #Reemplazando porque en el data coloque los nombres completos de las plataformas
    if platform=='disney':
        platform='disney plus'
  
    if platform=='amazon':
        platform='amazon prime'


   # Verificar que el score esté dentro del rango válido
    if scored is not None and (scored <= 3.34 or scored >= 3.72):
        raise ValueError("El score es una promedio por lo que debe estar dentro del rango de 3.34 a 3.72.")

   # Filtrar las películas para la plataforma, año y puntaje especificados
    df_soloPeliculasFiltradas = df[(df.platform == platform) & (df.score > scored) & (df.release_year == year) & (df.type == 'movie')]
   
   # Verificar que hay al menos una película que cumpla con los filtros
    if df_soloPeliculasFiltradas.empty:
        raise ValueError("No existe una película que cumpla con los filtros especificados.")

    # Agrupar por plataforma y contar el número de filas resultantes
    count = df_soloPeliculasFiltradas.groupby('platform').size()
    
    return count.to_dict()

In [ ]:
get_score_count('netflix', 3.35, 2020)


{'netflix': 516}

#Cantidad de películas por plataforma con filtro de PLATAFORMA. (La función debe llamarse get_count_platform(platform))

In [ ]:
def get_count_platform(platform=None):
    
    global df    
    
    # Verificar que la plataforma sea una de las opciones válidas
    if platform is not None and platform.lower() not in ['disney', 'amazon', 'hulu', 'netflix']:
        raise ValueError("La plataforma debe ser una de las opciones válidas: Disney, Amazon, Hulu o Netflix.")
    
    #Reemplazando porque en el data coloque los nombres completos de las plataformas
    if platform=='disney':
        platform='disney plus'
  
    if platform=='amazon':
        platform='amazon prime'
  
    # Filtrar las películas para la plataforma especificada
    df_soloPeliculasFiltradas = df[(df.platform == platform) & (df.type == 'movie')]

    # Agrupar por plataforma y contar el número de filas resultantes
    count = df_soloPeliculasFiltradas.groupby('platform').size()
    
    # Verificar que hay al menos una película que cumpla con los filtros
    if df_soloPeliculasFiltradas.empty:
        raise ValueError("No hay peliculas para esa plataforma")
    
    return count

In [ ]:
get_count_platform('disney')

platform
disney plus    1052
dtype: int64

#Actor que más se repite según plataforma y año. (La función debe llamarse get_actor(platform, year)

In [ ]:
def get_actor(platform=None, year=None):
    
    global df
    
    # Verificar que la plataforma sea una de las opciones válidas
    if platform is not None and platform.lower() not in ['disney', 'amazon', 'hulu', 'netflix']:
        raise ValueError("La plataforma debe ser una de las opciones válidas: Disney, Amazon, Hulu o Netflix.")
    
    #Reemplazando porque en el data coloque los nombres completos de las plataformas
    if platform=='disney':
        platform='disney plus'
  
    if platform=='amazon':
        platform='amazon prime'

    # Verificar que el año esté dentro del rango válido
    if year is not None and (year <= 1920 or year >= 2021):
        raise ValueError("El año debe estar dentro del rango de 1920 a 2021.")

    # Filtrar las películas para la plataforma  y año especificado
    df_plataformaAño = df[(df.platform == platform) & (df.release_year == year)]
    
           
    # Poner el cast en un array para poder hacer el recorrido
    df_castPlataformaAño= df_plataformaAño.assign(actor=df_plataformaAño.cast.str.split(',')).explode('cast')
    
    # Cantidad de apariciones de cada actor
    actor_counts = df_castPlataformaAño.cast.value_counts()
    
    # Obtener el actor que más se repite y su cantidad de apariciones
    max_actor = actor_counts.index[0]
    max_count = actor_counts.iloc[0]
    
    # Devolver el nombre del actor y su cantidad de apariciones
    return max_actor, max_count


In [ ]:
get_actor('netflix',2000)

('eddie murphy, janet jackson, larry miller, john ales, richard gant, anna maria horsford, melinda mcgraw, jamal mixon, gabriel williams, chris elliott',
 1)

# Codigo reacondicionado para FastAPi

In [ ]:
from typing import Optional
from fastapi import FastAPI
import pandas as pd
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder


app = FastAPI()

#Se define el data que se usará en las consultas
df = pd.read_csv('df.csv')

#consulta 1
#Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN
@app.get('/get_max_duration({release_year},{platform},{duration_type})')
def get_max_duration(release_year: Optional[int] = None, platform: Optional[str] = None, duration_type: Optional[str] = None):
     
    global df    
    
    # Verificar que el año esté dentro del rango válido
    if release_year is not None and (release_year <= 1920 or release_year >= 2021):
        raise ValueError("El año debe estar dentro del rango de 1920 a 2021.")

    # Verificar que la plataforma sea una de las opciones válidas
    if platform is not None and platform.lower() not in ['disney', 'amazon', 'hulu', 'netflix']:
        raise ValueError("La plataforma debe ser una de las opciones válidas: Disney, Amazon, Hulu o Netflix.")
   
    #Reemplazando porque en el data coloque los nombres completos de las plataformas
    if platform=='disney':
        platform='disney plus'
  
    if platform=='amazon':
        platform='amazon prime'

    # Verificar que el duracion_type esté dentro del rango válido
    if duration_type is not None and duration_type.lower() not in ['min', 'season', 'seasons']:
        raise ValueError("La duración debe ser una de las opciones válidas: min, season, seasons.")

    # Filtrar por películas únicamente
    df_soloPeliculas = df[df.type == 'movie']
    
    # Aplicar filtros opcionales si se proporcionan
    if release_year:
         df_soloPeliculas = df_soloPeliculas[df_soloPeliculas.release_year == release_year]
    if platform:
        df_soloPeliculas = df_soloPeliculas[df_soloPeliculas.platform == platform]
    if duration_type:
        df_soloPeliculas = df_soloPeliculas[df_soloPeliculas.duration_type == duration_type]

    # Verificar que hay al menos una película que cumpla con los filtros
    if df_soloPeliculas.empty:
        raise ValueError("No existe una película que cumpla con los filtros especificados.")

    # Ordenar por duración descendente y tomar la primera fila
    peliculaConMayorDuracion = df_soloPeliculas.sort_values('duration_type', ascending=False).iloc[0]

    detalles_peliculaConMayorDuracion = {
        'titulo': peliculaConMayorDuracion.title,
        'year': int(peliculaConMayorDuracion.release_year), # se tiene que agregar el int porque el formato int64 no permitia la serializacion del diccionario
        'platform': peliculaConMayorDuracion.platform,
        'type': peliculaConMayorDuracion.type,
        'duracion': peliculaConMayorDuracion.duration_type}

    return JSONResponse(content=jsonable_encoder(detalles_peliculaConMayorDuracion))



#consulta2
#Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año (la función debe llamarse get_score_count(platform, scored, year))
@app.get('/get_score_count({platform},{scored},{release_year})')
def get_score_count(platform: str =None, scored:float=None, release_year:int=None):
    
    global df    

    # Verificar que el año esté dentro del rango válido
    if release_year is not None and (release_year <= 1920 or release_year >= 2021):
        raise ValueError("El año debe estar dentro del rango de 1920 a 2021.")

    # Verificar que la plataforma sea una de las opciones válidas
    if platform is not None and platform.lower() not in ['disney', 'amazon', 'hulu', 'netflix']:
        raise ValueError("La plataforma debe ser una de las opciones válidas: Disney, Amazon, Hulu o Netflix.")
   
    #Reemplazando porque en el data coloque los nombres completos de las plataformas
    if platform=='disney':
        platform='disney plus'
  
    if platform=='amazon':
        platform='amazon prime'

   # Verificar que el score esté dentro del rango válido, error de cero y cinco
    if scored is not None and (scored <= 3.34 or scored >= 3.72):
        raise ValueError("El score es una promedio por lo que debe estar dentro del rango de 3.34 a 3.72.")
    
   # Filtrar las películas para la plataforma, año y puntaje especificados
    df_soloPeliculasFiltradas = df[(df.platform == platform) & (df.score > scored) & (df.release_year == release_year) & (df.type == 'movie')]

   # Verificar que hay al menos una película que cumpla con los filtros
    if df_soloPeliculasFiltradas.empty:
        raise ValueError("No existe una película que cumpla con los filtros especificados.")

    # Agrupar por plataforma y contar el número de filas resultantes
    count = df_soloPeliculasFiltradas.groupby('platform').size()
    
    return count.to_dict()


#Consulta3
#Cantidad de películas por plataforma con filtro de PLATAFORMA. (La función debe llamarse get_count_platform(platform))
@app.get('/get_count_platform({platform})')
def get_count_platform(platform=None):
    
    global df    
    
    # Verificar que la plataforma sea una de las opciones válidas
    if platform is not None and platform.lower() not in ['disney', 'amazon', 'hulu', 'netflix']:
        raise ValueError("La plataforma debe ser una de las opciones válidas: Disney, Amazon, Hulu o Netflix.")
    
    #Reemplazando porque en el data coloque los nombres completos de las plataformas
    if platform=='disney':
        platform='disney plus'
  
    if platform=='amazon':
        platform='amazon prime'
  
    # Filtrar las películas para la plataforma especificada
    df_soloPeliculasFiltradas = df[(df.platform == platform) & (df.type == 'movie')]

    # Agrupar por plataforma y contar el número de filas resultantes
    count = df_soloPeliculasFiltradas.groupby('platform').size()
    
    # Verificar que hay al menos una película que cumpla con los filtros
    if df_soloPeliculasFiltradas.empty:
        raise ValueError("No hay peliculas para esa plataforma")
    
    return count.to_dict()



#consulta 4
#Actor que más se repite según plataforma y año. (La función debe llamarse get_actor(platform, year)
@app.get('/get_actor/{platform},{year}')
def get_actor(platform: str = None, year: int = None):
    
    global df
    
    # Verificar que la plataforma sea una de las opciones válidas
    if platform is not None and platform.lower() not in ['disney', 'amazon', 'hulu', 'netflix']:
        raise ValueError("La plataforma debe ser una de las opciones válidas: Disney, Amazon, Hulu o Netflix.")
    
    #Reemplazando porque en el data coloque los nombres completos de las plataformas
    if platform=='disney':
        platform='disney plus'
  
    if platform=='amazon':
        platform='amazon prime'

    # Verificar que el año esté dentro del rango válido
    if year is not None and (year <= 1920 or year >= 2021):
        raise ValueError("El año debe estar dentro del rango de 1920 a 2021.")

    # Filtrar las películas para la plataforma y año especificado
    df_plataformaAño = df[(df.platform == platform) & (df.release_year == year)]
         
    # Poner el cast en un array para poder hacer el recorrido
    df_castPlataformaAño= df_plataformaAño.assign(actor=df_plataformaAño.cast.str.split(',')).explode('cast')
    
    # Contar la cantidad de apariciones de cada actor
    actor_counts = df_castPlataformaAño.cast.value_counts()
    
    # Obtener el actor que más se repite y su cantidad de apariciones
    max_actor = actor_counts.index[0]
    max_count = int(actor_counts.iloc[0])
    
    # Creando un diccionario para poder ver los resultados 
    actorfamoso = {'actor': max_actor, 'count': max_count}
    return JSONResponse(content=jsonable_encoder(actorfamoso))

<p align="right">🙇‍♀️</p>